In [1]:
from scrapy import Spider
from scrapy.selector import Selector
from scrapy.item import Item, Field
import pandas as pd
import calendar
import requests
import numpy as np
from datetime import datetime
from urllib.request import urlopen
from bs4 import BeautifulSoup
from threading import Timer
import time
import schedule

In [2]:
def extract_df(link):
    url = "https://markets.businessinsider.com/bonds"
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    response = requests.get(link)      #requests.get()
    doc = BeautifulSoup(response.text, 'html.parser')  #Now 'doc' contains entire html in parsed format
    
    tr_parent1 = doc.find_all('tbody',{'class':'table__tbody'}) 
    headers=['isin','name', 'currency', 'coupon', 'yield', "moody's rating", 'maturity date', 'bid', 'ask']
    data_dict = {}
    for i in range(len(headers)):
        data_dict[headers[i]] = []
        
    i = 0
    num_bonds = int(len(tr_parent1[0].find_all('td')))/8
    storage = []
    while True and i/8 != num_bonds:
        if i%8 == 0:
            isin_data = tr_parent1[0].find_all('td')
            isin = str(isin_data[i]).split('-')[-1].split('"')[0]
            storage.append(isin)
        data = tr_parent1[0].find_all('td',{'class':"table__td"})
        extracted_data = data[i].text.strip()   
        storage.append(extracted_data)
        i+=1

    for i in range(len(storage)):
        data_dict[headers[i%9]].append(storage[i])
        
    df = pd.DataFrame(data_dict)
    df[['bid','ask']] = df[['bid','ask']].replace('-','NaN').astype(float)
    df = df.drop(columns=['name','currency',"moody's rating"])
    df['close'] = (df['bid']+df['ask'])/2

    storage = []
    for i in range(df.shape[0]):
        yield_value = df['yield'].values[i].split('%')[0][:3]
        coupon = df['coupon'].values[i].split('%')[0][:3]
        df['maturity date'] = pd.to_datetime(df['maturity date'])
        mth = calendar.month_name[df.iloc[i]['maturity date'].month][:3]
        day = df.iloc[i]['maturity date'].day
        yr = df.iloc[i]['maturity date'].year
        new_col = 'CAN' + ' ' + str(coupon) + ' '+ str(mth) + ' ' + str(day) + ' ' + str(yr)
        storage.append(new_col)
    df = df.drop(columns=['coupon', 'maturity date', 'bid', 'ask'])
    df['name'] = np.array(storage)
    df = df.set_index('name')
    return df


def get_daily_df(home_urls):    
    for i in range(3):
        globals()['df%s' %i] = extract_df(home_urls[i])

    today = datetime.now()
    name_of_file = str(today.strftime("%d"))+str(today.strftime("%b"))
    dfs = [df0, df1, df2]
    pd.concat(dfs).to_json(f'APM466/{name_of_file}.json')


In [3]:
home_urls = ['https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=shortterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19',
             'https://markets.businessinsider.com/bonds/finder?p=2&borrower=71&maturity=shortterm&bondtype=2%2c3%2c4%2c16&currency=184&country=19',
             'https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=midterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19']


In [4]:
get_daily_df(home_urls)